# Training on plain or encrypted images

## 1. Downloading the plain sample dataset

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown https://drive.google.com/uc?id=1iq9NkenkTgpYNLYYb0fXe1aQpDZ_iFt6


Downloading...
From: https://drive.google.com/uc?id=1iq9NkenkTgpYNLYYb0fXe1aQpDZ_iFt6
To: /content/Plain-4-classes-sample20.zip
100% 2.96M/2.96M [00:00<00:00, 198MB/s]


In [ ]:
!mkdir -p '/content/drive/My Drive/COVID-sample-dataset'

In [ ]:
!unzip /content/Plain-4-classes-sample20.zip -d '/content/drive/My Drive/COVID-sample-dataset'

### Paths of the plain and encrypted dataset (to be generated)

In [ ]:
path_in = '/content/drive/My Drive/COVID-sample-dataset/Plain-4-classes-sample20/' # path of the plain dataset containing 1 subfolder for each class
path_out = '/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20' # folder path for the encrypted dataset to be generated

The encryption of the dataset takes time (about 30s per image). If you want to skip this step, you can download the already encrypted dataset below, then move on to step 3 (Training on plain images):

In [ ]:
!gdown https://drive.google.com/uc?id=1GG_VASj76TbUddzT1qqRX9OKIL4BrF5D
!unzip /content/Encrypted-4-classes-sample20.zip -d '/content/drive/My Drive/COVID-sample-dataset'

Downloading...
From: https://drive.google.com/uc?id=1GG_VASj76TbUddzT1qqRX9OKIL4BrF5D
To: /content/Encrypted-4-classes-sample20.zip
100% 7.12M/7.12M [00:00<00:00, 40.9MB/s]
Archive:  /content/Encrypted-4-classes-sample20.zip
replace /content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Lung_Opacity/Lung_Opacity-8.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Lung_Opacity/Lung_Opacity-8.png  
  inflating: /content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Lung_Opacity/Lung_Opacity-2.png  
  inflating: /content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Lung_Opacity/Lung_Opacity-19.png  
  inflating: /content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Lung_Opacity/Lung_Opacity-16.png  
  inflating: /content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Lung_Opacity/Lung_Opacity-9.png  
  inflating: 

## 2. Encryption of plain images

In [ ]:
import numpy
from PIL import Image 
import numpy as np
from skimage.io import imsave
import os
import glob

In [ ]:
# Properties of Partially Homomorphic Encryption
def lcm(a, b):
    """
    lcm(a, b)
    returns Lowest Common Multiple of a and b
    """
    return a * b // xgcd(a,b)[0]

def xgcd(a, b):
    """
    xgcd (a, b)
    returns (g, x, y) according to the Extended Euclidean Algorithm
    such that, ax + by = g
    """
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = xgcd(b % a, a)
        return (g, x - (b // a) * y, y)

def multiplicative_inverse(a, modulus):
    """
    multiplicative_inverse(a, modulus)
    returns x: multiplicative inverse of a
    such that, a * x = 1 (mod modulus)
    """
    g, x, y = xgcd(a, modulus)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % modulus


def binary_exponent(base, exponent, modulus):
    """
    modular_binary_exponent( base, exponent, modulus)
    
    args:
        base
        exponent
        modulus
    
    generates:
        base ^ exponent (mod modulus)
    
        along with intermediate results from binary exponentiation
        required for Rabin-Miller primality test
    """
    if modulus == 1:
        yield 0
        return
    bitmask = 1 << exponent.bit_length() - 1
    res = 1
    while bitmask:
        res = (res * res) % modulus
        if bitmask & exponent:
            res = (res * base) % modulus
        yield res
        bitmask >>= 1

In [ ]:
# Properties of Partially Homomorphic Encryption
import random
#import ModularArithmetic as mod

def is_probably_prime(n):
    """
    is_probably_prime(n)
    args:
        n
    returns:
        Boolean
    returns True if n is a probable prime
    based on the Rabin-Miller primality test
    """
    tests = max(64, n.bit_length())
    for i in range(tests):
        rand = random.randint(1,n-1)
        return 1 in binary_exponent(rand, n-1, n)

def generate_prime(bitlen=128):
    """
    generate_prime(bitlen)
    args:
        bitlen: length of number to be generated (default: 128)
    returns:
        a probable prime of bitlen bits
        based on the Rabin-Miller primality test
    """
    n = random.getrandbits(bitlen) | 1<<(bitlen-1) | 1
    while not is_probably_prime(n):
        n = random.getrandbits(bitlen) | 1<<(bitlen-1) | 1
    return n

In [ ]:
# Properties of Partially Homomorphic Encryption
import random

#import ModularArithmetic
#import RabinMiller

class PrivateKey:
    """
    PrivateKey object contains λ and μ
    in accordance to the Paillier Cryptosystem
    
    args:
        p: a prime number
        q: another prime number
        (p and q are of equal length)
        n: product of p and q
        
    attributes:
        λ: lowest common multiple of p-1 and q-1
        ∵ p and q are of equal length we can use the simplification,
        μ: modular multiplicative inverse of λ and n
    """
    
    def __init__(self, p, q, n):

        self.λ = lcm( p-1, q-1)
        self.μ = multiplicative_inverse( self.λ, n)
        
    def __repr__(self):
        return ("---\nPrivate Key :\nλ:\t"+str(self.λ) +"\nμ:\t"+str(self.μ) +"\n---")


class PublicKey:
    """
    Public Key object contains n and g
    in accordance to the Paillier Cryptosystem
    
    args:
        n: product of two equal lenght prime numbers
    
    attributes:
        n: product of two primes
        g: a random number such that,
        multiplicative order of g in n^2 is a multiple of n
        
        ∵ p and q are of equal length we can use a simplification of g = n+1
    """
    def __init__(self, n):
        self.n = n
        self.nsq = n * n
        self.g = n+1
    
    def __repr__(self):
        return ("---\nPublic Key :\nn:\t"+ str(self.n) +"\n---")


def generate_keys(bitlen):
    """
    generate_keys( bitlen)
    
    args:
        bitlen: length of primes to be generated (default: 128)
    
    returns Public Private key pair as a tuple
    (PublicKey, PrivateKey)
    """
    
    p = generate_prime(bitlen)
    q = generate_prime(bitlen)
    n = p * q
    return (PublicKey(n), PrivateKey(p, q, n))

In [ ]:
# Properties of Somewhat Homomorphic Encryption
from PIL import Image
from PIL import Image
import numpy as np
# defining the big O constant
#bigOconstant = 2
#_lambda = 2
# Function to calculate eta value from the security parameter lambda
def eta(_lambda):			# eta is bit length of secret key 
	return (bigOconstant * _lambda * _lambda)
# Function to calculate gamma value from the security parameter lambda
def gamma(_lambda):
	return (bigOconstant * _lambda**5)
# Function to calculate tau value from the security parameter lambda
def tau(_lambda):			# tau is number of integers in public key
	return (gamma(_lambda) + _lambda)
# Function to calculate rho' value from security parameter lambda
def rho_dash(_lambda):
	return (2*_lambda)
# Function to generate a large value
# check_lsb = 1 if the number has to be odd and check msb = 1 to set the initial bit to be 1
def number_generator(num_of_bits, check_msb, check_lsb):
	number_count = num_of_bits / 32
	number_count = int(number_count)
	base = 2**32
	rand_num ={}
	if (number_count == 0):
		x = int(numpy.random.uniform(0,2**num_of_bits))
	else :
		for i in range(number_count):
			rand_num[i] = int(numpy.random.uniform(0,2**32))
		if (check_lsb == 1):
			rand_num[0] = ((rand_num[0] & ~1 ) + 1)
		if (check_msb == 1) :
			rand_num[number_count-1] = ((rand_num[number_count-1] & ~2**31 ) + 2**31)
		x=0
	# number = x0*base**0 + x1*base**1 + x2*base**2 + x3*base**3 + ...
		for i in range(number_count):
			x +=  rand_num[i] * base**i
	return x

# Function to calculate the secret key which is an odd eta-bit integer
def secret_key(_lambda):
	eta_value = eta(_lambda)
	secret_key = number_generator(eta_value, 1, 1)
	return secret_key	
# Function to find the different values of public key
def pubkey_distribution(_lambda, secret_key):
	#q_bound = int(math.ceil(math.log((2**gamma(_lambda) / secret_key),2)))
	#q = number_generator(q_bound, 0)
	q = int(numpy.random.uniform(0,2**256))
	r = (1 - 2*(numpy.random.randint(0,2))) * number_generator(_lambda, 0, 0)
	x = secret_key * q + r
	return x
	
# Function to calculate the public key list
def public_keyS(_lambda, secret_key):
	while True:
		pubkey = [pubkey_distribution(_lambda, secret_key) for i in range(tau(_lambda))]
		pubkey_max = max(pubkey)	# get largest value from list	
		pubkey_mod_sk = pubkey_max % secret_key
		# loop till largest value is odd and the value modulus secret key is even
		if (pubkey_max & 1) and (pubkey_mod_sk % 2 == 0):
			break
	return pubkey

# Function to generate a random subset S from {1, 2, 3, .... tau}
def generate_random_subset(_tau):
	set_size = numpy.random.randint(1, _tau)
	S = set()
	while len(S) < set_size:
		x = numpy.random.randint(1, _tau)
		S.add(x)
	return S
# Properties of Somewhat Homomorphic Encryption
# Function to encrypt one value(Hybrid Encryption function combines PHE and SHE)	
def Hybrid_encryption(_lambda, public_keyS, plaintext, public_key):
    """
		Hybrid_encryption(_lambda, public_keyS, plaintext, public_key)
			args:
				_lambda : SHE security parameter lambda
				public_keyS: SHE public_keyS function 
				plaintext : number to be encrypted
				public_key : Paillier Publickey object
			returns:
				c: encryption of plaintext
				such that c = (plaintext + 2*r + 2*sum_x) (mod n ^ 2)
				where, r(PHE scheme) is a random number in n(PHE scheme) such that r and n are coprime and 
				sum_x(SHE scheme) is the summation of xi's for i belongs to S i.e subset of {1,2,3 ... tau}
    """
    r = random.randint( 1, public_key.n-1)
    while not xgcd( r, public_key.n)[0] == 1:
        r = random.randint( 1, public_key.n)   
    random_subset = generate_random_subset(tau(_lambda))
    sum_x = 0
	# the summation of xi's for i belongs to S i.e subset of {1,2,3 ... tau}
    for i in random_subset:
        sum_x += public_keyS[i]
    c = (plaintext + 2*r + 2*sum_x) % public_key.nsq 
    return c

In [ ]:
# Function call to encrypt image
def ImgEncrypt(_lambda, pk, plainimg, pub):
    """
	ImgEncrypt(_lambda, pk, plainimg, pub)
		args:
			_lambda : SHE security parameter lambda
			pk : SHE public_keyS function 
			plainimg : PIL Image object
			pub : Paillier Publickey object
		returns:
			cipherimg: Encryption of plainimg
    """
    cipherimg = np.asarray(plainimg)
    shape = cipherimg.shape
    cipherimg = cipherimg.flatten().tolist()
    cipherimg = [Hybrid_encryption(_lambda, pk, pix, pub) for pix in cipherimg]
    
    return np.asarray(cipherimg).reshape(shape)

In [ ]:
import time
start = time.time()
bigOconstant = 2
_lambda = 2
# function call to generate secret key(Properties of Somewhat Homomorphic Encryption)
print('Generating secret key...')
sk = secret_key(_lambda)
# function call to generate public key list(Properties of Somewhat Homomorphic Encryption)		
print('Generating public key (if it takes too much time, stop and rerun the cell to start from a new generation)...')
pk = public_keyS(_lambda, sk)
# function call to generate tuple key Public Private key (Properties of Partially Homomorphic Encryption)		
print('Generating Public/Private keys for PHE...')
pub, priv = generate_keys(bitlen=128) 

if not os.path.exists(path_out):
    os.makedirs(path_out)

# print('glob.glob(os.path.join(path_in, "*"))=',glob.glob(os.path.join(path_in, "*")))
for class_folder in glob.glob(os.path.join(path_in, "*")):
    print("Folder: ", class_folder)
    print("------------------------------------------------------------------------")
    encr_folder = os.path.join(
                path_out,
                os.path.basename(class_folder))
    if not os.path.exists(encr_folder):
        os.makedirs(encr_folder)
    for filename in glob.glob(os.path.join(class_folder,'*.png')): 
        with open(os.path.join(os.getcwd(), filename), 'r+') as f:
            print(filename)
#             a1 = filename
#             try:
            plainimg = Image.open(filename)
            cipherimg = ImgEncrypt(_lambda, pk, plainimg, pub)
            arra = np.array(cipherimg, dtype = 'float')
            encr_filepath = os.path.join(
                encr_folder,
                os.path.basename(filename)
            )
            imsave(encr_filepath,arra) #.astype(numpy.uint8))
            print(encr_filepath, 'saved.')
#             except:
#                 print("Pb in: ", filename)

Generating secret key...
Generating public key (if it takes too much time, stop and rerun the cell to start from a new generation)...
Generating Public/Private keys for PHE...
Folder:  /content/drive/My Drive/COVID-sample-dataset/Plain-4-classes-sample20/Covid
------------------------------------------------------------------------
/content/drive/My Drive/COVID-sample-dataset/Plain-4-classes-sample20/Covid/COVID-1.png


/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Covid/COVID-1.png saved.
/content/drive/My Drive/COVID-sample-dataset/Plain-4-classes-sample20/Covid/COVID-2.png


/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Covid/COVID-2.png saved.
/content/drive/My Drive/COVID-sample-dataset/Plain-4-classes-sample20/Covid/COVID-3.png


/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Covid/COVID-3.png saved.
/content/drive/My Drive/COVID-sample-dataset/Plain-4-classes-sample20/Covid/COVID-4.png


/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Covid/COVID-4.png saved.
/content/drive/My Drive/COVID-sample-dataset/Plain-4-classes-sample20/Covid/COVID-5.png


...

/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Viral_Pneumonia/Viral Pneumonia-16.png saved.
/content/drive/My Drive/COVID-sample-dataset/Plain-4-classes-sample20/Viral_Pneumonia/Viral Pneumonia-17.png


/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Viral_Pneumonia/Viral Pneumonia-17.png saved.
/content/drive/My Drive/COVID-sample-dataset/Plain-4-classes-sample20/Viral_Pneumonia/Viral Pneumonia-18.png


/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Viral_Pneumonia/Viral Pneumonia-18.png saved.
/content/drive/My Drive/COVID-sample-dataset/Plain-4-classes-sample20/Viral_Pneumonia/Viral Pneumonia-19.png


/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Viral_Pneumonia/Viral Pneumonia-19.png saved.
/content/drive/My Drive/COVID-sample-dataset/Plain-4-classes-sample20/Viral_Pneumonia/Viral Pneumonia-20.png


/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Viral_Pneumonia/Viral Pneumonia-20.png saved.


In [ ]:

!ls -R '/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/' 

'/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/':
Covid  Lung_Opacity  Normal  Viral_Pneumonia

'/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Covid':
COVID-10.png  COVID-14.png  COVID-18.png  COVID-2.png  COVID-6.png
COVID-11.png  COVID-15.png  COVID-19.png  COVID-3.png  COVID-7.png
COVID-12.png  COVID-16.png  COVID-1.png   COVID-4.png  COVID-8.png
COVID-13.png  COVID-17.png  COVID-20.png  COVID-5.png  COVID-9.png

'/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/Lung_Opacity':
Lung_Opacity-10.png  Lung_Opacity-17.png  Lung_Opacity-4.png
Lung_Opacity-11.png  Lung_Opacity-18.png  Lung_Opacity-5.png
Lung_Opacity-12.png  Lung_Opacity-19.png  Lung_Opacity-6.png
Lung_Opacity-13.png  Lung_Opacity-1.png   Lung_Opacity-7.png
Lung_Opacity-14.png  Lung_Opacity-20.png  Lung_Opacity-8.png
Lung_Opacity-15.png  Lung_Opacity-2.png   Lung_Opacity-9.png
Lung_Opacity-16.png  Lung_Opacity-3.png

'/content/drive/My Drive/CO

## 3. Training on plain images

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from glob import glob
from PIL import Image
import os
import random
import cv2
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (
    Activation,
    Dense,
    Dropout,
    Flatten,
    Conv2D,
    MaxPooling2D,
    MaxPool2D,
    AveragePooling2D,
    GlobalMaxPooling2D,
)
from tensorflow.keras import backend as K
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical  # convert to one-hot-encoding
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import PrecisionAtRecall, Recall

from tensorflow.keras.applications.efficientnet import EfficientNetB1
from tensorflow.keras.applications.nasnet import NASNetLarge
from tensorflow.keras.applications import DenseNet201, MobileNetV2, Xception, ResNet152V2, InceptionV3, VGG19, DenseNet121, ResNet50, VGG16, DenseNet169

from tensorflow.keras.layers import AveragePooling2D, Conv2D, MaxPool2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tensorflow.keras import initializers
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_recall_fscore_support,
)
import tensorflow.keras as keras
from tensorflow.keras.callbacks import CSVLogger
import argparse


np.random.seed(0)
tf.random.set_seed(0)

### 3.1 Choice of model and hyerparameters
The ```encryption_status``` variable specifies whether the training will be done on plain or encrypted images.

In [ ]:
dict_models = {'EfficientNetB1': EfficientNetB1, 
                'NASNetLarge': NASNetLarge,
                'DenseNet201': DenseNet201,
                'MobileNetV2': MobileNetV2,
                'Xception': Xception,
                'VGG19': VGG19,
                'VGG16': VGG16,
                'ResNet152V2': ResNet152V2,
                'InceptionV3': InceptionV3,
                'ResNet50': ResNet50,
                'DenseNet121': DenseNet121,
                'DenseNet169': DenseNet169}

model_name = 'ResNet50'
IMG_SHAPE = 224
INIT_LR = 1e-4
INITIAL_EPOCH = 0
batch_size = 8
epochs = 250
fine_tune_at = 100



### 3.2 Paths

In [ ]:
base_path = '/content/drive/My Drive/COVID-sample-dataset/Plain-4-classes-sample20/'

CHECKPOINT_PATH_BASE = "/content/drive/My Drive/COVID-sample-dataset/Plain-checkpoints-" + model_name

if not os.path.exists(CHECKPOINT_PATH_BASE):
    os.makedirs(CHECKPOINT_PATH_BASE)

CHECKPOINT_PATH = CHECKPOINT_PATH_BASE+'/Plain-'+model_name+"-epoch{epoch:02d}-loss{val_loss:.2f}-val_acc{val_accuracy:.3f}.h5"
LOGFILE_PATH = CHECKPOINT_PATH_BASE+'/Plain-'+model_name+"-training-log.csv"



### 3.3 Data augmentation and data generation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2,
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    base_path,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    # shuffle=False,
    seed=0,
)

validation_generator = test_datagen.flow_from_directory(
    base_path,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    # shuffle=False,
    seed=0,
)

Found 64 images belonging to 4 classes.
Found 15 images belonging to 4 classes.


### 3.4 Definition of the CNN model and compilation

In [ ]:
baseModel = dict_models[model_name](
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SHAPE, IMG_SHAPE, 3)
    # input_tensor=Input(shape=IMG_SHAPE),
)
# construct the head of the model that will be placed on top of the
# the base model
baseModel.trainable = True

model = tf.keras.Sequential(
    [
        baseModel,
        tf.keras.layers.Conv2D(128, 3, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(4, activation="softmax"),
    ]
)

cp1 = ModelCheckpoint(
    filepath=CHECKPOINT_PATH,
    monitor='val_accuracy',#"val_loss",
    save_best_only=True,
    verbose=1,
    mode="auto",
)
csv_logger = CSVLogger(LOGFILE_PATH, append=True)
callbacks_list = [cp1, csv_logger]


steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

for layer in baseModel.layers[:fine_tune_at]:
  layer.trainable =  False


model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(INIT_LR), # or: RMSprop
              metrics=['accuracy'])



### 3.5 Start training

In [ ]:
history_fine = model.fit_generator(train_generator, 
                                   steps_per_epoch = steps_per_epoch,
                                   epochs=epochs, 
                                   workers=4,
                                   validation_data=validation_generator, 
                                   validation_steps=validation_steps,
                                   callbacks=callbacks_list)


Epoch 1/250


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


8/8 [==============================] - ETA: 0s - loss: 3.4031 - accuracy: 0.2188
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to /content/drive/My Drive/COVID-sample-dataset/Plain-checkpoints-ResNet50/Plain-ResNet50-epoch01-loss2.11-val_acc0.500.h5
8/8 [==============================] - 12s 778ms/step - loss: 3.4031 - accuracy: 0.2188 - val_loss: 2.1105 - val_accuracy: 0.5000
Epoch 2/250
8/8 [==============================] - ETA: 0s - loss: 1.8255 - accuracy: 0.1250
Epoch 2: val_accuracy did not improve from 0.50000
8/8 [==============================] - 3s 308ms/step - loss: 1.8255 - accuracy: 0.1250 - val_loss: 1.7302 - val_accuracy: 0.0000e+00
Epoch 3/250
8/8 [==============================] - ETA: 0s - loss: 1.6427 - accuracy: 0.0938    
Epoch 3: val_accuracy did not improve from 0.50000
8/8 [==============================] - 3s 289ms/step - loss: 1.6427 - accuracy: 0.0938 - val_loss: 1.7758 - val_accuracy: 0.0000e+00
Epoch 4/250
8/8 [=========================

KeyboardInterrupt: ignored

In [ ]:
# Check the saved checkpoints:
print(CHECKPOINT_PATH_BASE)
os.listdir(CHECKPOINT_PATH_BASE)

/content/drive/My Drive/COVID-sample-dataset/Plain-checkpoints-ResNet50


['-epoch01-loss1.90-val_acc0.250.h5',
 '-training-log.csv',
 '-epoch02-loss1.38-val_acc0.375.h5',
 '-epoch30-loss1.57-val_acc0.500.h5',
 '-epoch33-loss0.86-val_acc0.625.h5',
 '-epoch40-loss0.80-val_acc0.875.h5',
 'Plain-ResNet50-epoch01-loss1.70-val_acc0.250.h5',
 'Plain-ResNet50-training-log.csv',
 'Plain-ResNet50-epoch07-loss1.14-val_acc0.375.h5',
 'Plain-ResNet50-epoch08-loss1.37-val_acc0.500.h5',
 'Plain-ResNet50-epoch41-loss2.12-val_acc0.625.h5',
 'Plain-ResNet50-epoch67-loss1.57-val_acc0.750.h5',
 'Plain-ResNet50-epoch01-loss2.11-val_acc0.500.h5']

## 4. Training on encrypted images

### 4.1 Paths

In [ ]:
base_path = '/content/drive/My Drive/COVID-sample-dataset/Encrypted-4-classes-sample20/'

CHECKPOINT_PATH_BASE = "/content/drive/My Drive/COVID-sample-dataset/Encrypted-checkpoints-" + model_name

if not os.path.exists(CHECKPOINT_PATH_BASE):
    os.makedirs(CHECKPOINT_PATH_BASE)

CHECKPOINT_PATH = CHECKPOINT_PATH_BASE+'/Encrypted-'+model_name+"-epoch{epoch:02d}-loss{val_loss:.2f}-val_acc{val_accuracy:.3f}.h5"
LOGFILE_PATH = CHECKPOINT_PATH_BASE+'/Encrypted-'+model_name+"-training-log.csv"


### 4.2 Data augmentation and data generation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2,
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    base_path,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    # shuffle=False,
    seed=0,
)

validation_generator = test_datagen.flow_from_directory(
    base_path,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    # shuffle=False,
    seed=0,
)

Found 64 images belonging to 4 classes.
Found 15 images belonging to 4 classes.


### 4.3 Definition of the CNN model and compilation

In [ ]:
baseModel = dict_models[model_name](
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SHAPE, IMG_SHAPE, 3)
    # input_tensor=Input(shape=IMG_SHAPE),
)
# construct the head of the model that will be placed on top of the
# the base model
baseModel.trainable = True

model = tf.keras.Sequential(
    [
        baseModel,
        tf.keras.layers.Conv2D(128, 3, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(4, activation="softmax"),
    ]
)

cp1 = ModelCheckpoint(
    filepath=CHECKPOINT_PATH,
    monitor='val_accuracy',#"val_loss",
    save_best_only=True,
    verbose=1,
    mode="auto",
)
csv_logger = CSVLogger(LOGFILE_PATH, append=True)
callbacks_list = [cp1, csv_logger]


steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

for layer in baseModel.layers[:fine_tune_at]:
  layer.trainable =  False


model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(INIT_LR), # or: RMSprop
              metrics=['accuracy'])

### 4.4 Start training


In [ ]:
history_fine = model.fit_generator(train_generator, 
                                   steps_per_epoch = steps_per_epoch,
                                   epochs=epochs, 
                                   workers=4,
                                   validation_data=validation_generator, 
                                   validation_steps=validation_steps,
                                   callbacks=callbacks_list)


In [ ]:
# Check the saved checkpoints:
print(CHECKPOINT_PATH_BASE)
os.listdir(CHECKPOINT_PATH_BASE)

/content/drive/My Drive/COVID-sample-dataset/Encrypted-checkpoints-ResNet50


['Encrypted-ResNet50-epoch01-loss1.42-val_acc0.250.h5',
 'Encrypted-ResNet50-training-log.csv',
 'Encrypted-ResNet50-epoch09-loss1.36-val_acc0.375.h5',
 'Encrypted-ResNet50-epoch50-loss1.36-val_acc0.500.h5',
 'Encrypted-ResNet50-epoch232-loss1.38-val_acc0.625.h5']